<a href="https://colab.research.google.com/github/kylexu/learnPython/blob/main/AnimeGANv2_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

An inference notebook for [**AnimeGAN**](https://github.com/TachibanaYoshino/AnimeGAN),
Made by Asher Chan.

If you like what I'm doing you can tip me and follow on [**Patreon**](https://www.patreon.com/Asher_Chan).    &nbsp;&nbsp;&nbsp;&nbsp;     ![visitors](https://visitor-badge.glitch.me/badge?page_id=AnimeGANv3&left_color=SeaGreen&right_color=Gold)  

To start using this, run the cells with `Ctrl+F9` or "Runtime > Run All" 

In [ ]:
#@title This colab is distributed under the AnimeGAN license
li = """License

This repo is made freely available to academic and 
non-academic entities for non-commercial purposes such 
as academic research, teaching, scientific publications. 
Permission is granted to use the AnimeGAN given 
that you agree to my license terms. Regarding the 
request for commercial use, please contact us via 
email to help you obtain the authorization letter."""
print(li)


License

This repo is made freely available to academic and 
non-academic entities for non-commercial purposes such 
as academic research, teaching, scientific publications. 
Permission is granted to use the AnimeGAN given 
that you agree to my license terms. Regarding the 
request for commercial use, please contact us via 
email to help you obtain the authorization letter.


In [ ]:
#@title Install and download. Run once.
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VPAPI84qaPUCHKHJLHiMK7BP_JE66xNe' -O AnimeGAN_Hayao.onnx
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=17XRNQgQoUAnu6SM5VgBuhqSBO4UAVNI1' -O AnimeGANv2_Hayao.onnx
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=10rQfe4obW0dkNtsQuWg-szC4diBzYFXK' -O AnimeGANv2_Shinkai.onnx
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1X3Glf69Ter_n2Tj6p81VpGKx7U4Dq-tI' -O AnimeGANv2_Paprika.onnx

!pip -qq install opencv-python
!pip -qq install numpy
!pip -qq install onnxruntime-gpu

In [ ]:
#@title Define functions
#@markdown Select model version and run.
import onnxruntime as ort
import time, cv2, PIL
import numpy as np
from tqdm.notebook import tqdm

vid_form = ['.mp4','.avi','.webm']
device_name = ort.get_device()


if device_name == 'cpu':
    providers = ['CPUExecutionProvider']
elif device_name == 'GPU':
    providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']

model = 'AnimeGAN_Hayao' #@param ['AnimeGAN_Hayao','AnimeGANv2_Hayao','AnimeGANv2_Shinkai','AnimeGANv2_Paprika']
#load model
session = ort.InferenceSession(f'/content/{model}.onnx', providers=providers)

def post_precess(img, wh):
    img = (img.squeeze()+1.) / 2 * 255
    img = img.astype(np.uint8).clip(0, 255)
    img = cv2.resize(img, (wh[0], wh[1]))
    return img

def process_image(img, x32=True):
    h, w = img.shape[:2]
    if x32: # resize image to multiple of 32s
        def to_32s(x):
            return 256 if x < 256 else x - x%32
        img = cv2.resize(img, (to_32s(w), to_32s(h)))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)/ 127.5 - 1.0
    return img


def Convert(img, scale):
    x = session.get_inputs()[0].name
    y = session.get_outputs()[0].name
    fake_img = session.run(None, {x : img})[0]
    images = (np.squeeze(fake_img) + 1.) / 2 * 255
    images = np.clip(images, 0, 255).astype(np.uint8)
    output_image = cv2.resize(images, scale[::-1])
    return cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)
    

in_dir = '/content/in'
out_dir = f"/content/outputs"

#setup colab interface
from google.colab import files
import ipywidgets as widgets
from IPython.display import clear_output 
from IPython.display import display
import os
from glob import glob

def reset(p):
  with output_reset:
    clear_output()
  clear_output()
  process()
 
button_reset = widgets.Button(description="Upload")
output_reset = widgets.Output()
button_reset.on_click(reset)

 
def get_video(video, out_name, output_format='MP4V'):
  # load video
  vid = cv2.VideoCapture(video)
  vid_name = os.path.basename(video)
  total = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
  fps = vid.get(cv2.CAP_PROP_FPS)
  width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
  codec = cv2.VideoWriter_fourcc(*output_format)
  
  video_out = cv2.VideoWriter(out_name, codec, fps, (width, height))
  pbar = tqdm(total=total, )
  pbar.set_description(f"Making: {os.path.basename(video).rsplit('.', 1)[0] + '_AnimeGAN.mp4'}")
  while True:
    ret, frame = vid.read()
    if not ret:
        break
    frame = np.asarray(np.expand_dims(process_image(frame),0))
    x = session.get_inputs()[0].name
    y = session.get_outputs()[0].name
    fake_img = session.run(None, {x : frame})[0]
    fake_img = post_precess(fake_img, (width, height))
    video_out.write(cv2.cvtColor(fake_img, cv2.COLOR_BGR2RGB))
    pbar.update(1)
  pbar.close()
  vid.release()
  video_out.release()
 

def process(upload=True):
  os.makedirs(in_dir, exist_ok=True)
  %cd {in_dir}/
  %rm -rf {out_dir}/*
  os.makedirs(out_dir, exist_ok=True)
  in_files = sorted(glob(f'{in_dir}/*'))
  if (len(in_files)==0) | (upload):
    %rm -rf {in_dir}/*
    uploaded = files.upload()
    if len(uploaded.keys())<=0: 
      print('\nNo files were uploaded. Try again..\n')
      return
  
  # Process the uploaded image.
  in_files = sorted(glob(f'{in_dir}/*'))
  in_files = [ x for x in in_files if os.path.splitext(x)[-1] in vid_form]
  for v in tqdm(in_files):
    out_name = f"{out_dir}/{v.split('/')[-1].split('.')[0]}.mp4"
    get_video(v, out_name) 


In [ ]:
#@title Click to upload files and run inference. Results will be saved.
print('\nPress the button and pick some videos to upload\n')
process()


Press the button and pick some photos to upload

/content/in


Saving お花見.mp4 to お花見.mp4


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/285 [00:00<?, ?it/s]